This is the python notebook that we use to run the model on the COCO dataset. We use a different version of training annotations of the COCO Dataset to make it compatible with the MPII case so as to set the benchmarks. The second cell from top generates the json file in the format we need after taking as input the actual training annotations file for COCO dataset.

In [46]:
import time
import torch.optim
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import torch.utils.data as data
import torch.backends.cudnn as cudnn
import sys
import numpy as np
import json
import resnet
import math
import os

from PIL import Image
import torch.backends.cudnn as cudnn
from tqdm import tqdm

import cv2

In [31]:
#generate json in format that we need-> only run this once to create json
import pandas as pd
import json
with open('annotations/person_keypoints_val2017.json') as f:
    data = json.load(f)
data.pop('info', None)
data.pop('licenses', None)
images_data = pd.DataFrame(data['images'])
images_data = images_data[['file_name', 'height','width','id']]
images_data = images_data.rename(columns = {'id':'image_id'})
annotations_data = pd.DataFrame(data['annotations'])
annotations_data = annotations_data.loc[annotations_data.iscrowd == 0 ]
annotations_data = annotations_data[['segmentation', 'keypoints','image_id']]
final = pd.merge(images_data, annotations_data, on=['image_id'])
final = final.to_dict(orient = 'records')
with open('val_coco.json', 'w') as fp:
    json.dump(final,fp,indent = 2)
    
with open('annotations/person_keypoints_train2017.json') as f:
    data = json.load(f)
data.pop('info', None)
data.pop('licenses', None)
images_data = pd.DataFrame(data['images'])
images_data = images_data[['file_name', 'height','width','id']]
images_data = images_data.rename(columns = {'id':'image_id'})
annotations_data = pd.DataFrame(data['annotations'])
annotations_data = annotations_data.loc[annotations_data.iscrowd == 0 ]
annotations_data = annotations_data[['segmentation', 'keypoints','image_id']]
final = pd.merge(images_data, annotations_data, on=['image_id'])
final = final.to_dict(orient = 'records')
with open('train_coco.json', 'w') as fp:
    json.dump(final,fp,indent = 2)

In [47]:
class Atrous(nn.Module):
    def __init__(self, inplanes, planes, kernel_size, padding, d, BN):
        super(Atrous, self).__init__()
        self.conv = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                                            stride=1, padding=padding, dilation=d, bias=False)
        self.batch = BN(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.batch(self.conv(x))
        return self.relu(x)

    def _init_weight(self):
        for mod in self.modules():
            if isinstance(mod, nn.Conv2d):
                torch.nn.init.kaiming_normal_(mod.weight)
            elif isinstance(mod, nn.BatchNorm2d):
                mod.weight.data.fill_(1)
                mod.bias.data.zero_()
            elif isinstance(mod, nn.BatchNorm2d):
                mod.weight.data.fill_(1)
                mod.bias.data.zero_()

class wasp(nn.Module):
    def __init__(self, bb, output_stride, BN):
        super(wasp,self).__init__()
        inplanes = 2048
        if output_stride == 16:
            ds = [24, 18, 12,  6]
        elif output_stride == 8:
            ds = [48, 36, 24, 12]
        else: 
            print('build wasp error')

        self.a1 = Atrous(inplanes, 256, 1, padding=0, d=ds[0], BN=BN)
        self.a2 = Atrous(256, 256, 3, padding=ds[1], d=ds[1], BN=BN)
        self.a3 = Atrous(256, 256, 3, padding=ds[2], d=ds[2], BN=BN)
        self.a4 = Atrous(256, 256, 3, padding=ds[3], d=ds[3], BN=BN)

        self.g_avg_pool = nn.Sequential(
                            nn.AdaptiveAvgPool2d((1, 1)),
                            nn.Conv2d(inplanes, 256, 1, stride=1, bias=False),
                            nn.ReLU())

        self.conv1 = nn.Conv2d(1280, 256, 1, bias=False)
        self.conv2 = nn.Conv2d(256,256,1,bias=False)
        self.bn1 = BN(256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self._init_weight()

    def forward(self, x):
        #atrous layers
        x1 = self.a1(x);x2 = self.a2(x1);x3 = self.a3(x2);x4 = self.a4(x3)

        #conv layers
        x1 = self.conv2(x1);x2 = self.conv2(x2);x3 = self.conv2(x3);x4 = self.conv2(x4)
    
        x1 = self.conv2(x1);x2 = self.conv2(x2);x3 = self.conv2(x3);x4 = self.conv2(x4)

        x5 = self.g_avg_pool(x)
        x5 = F.interpolate(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat((x1, x2, x3, x4, x5), dim=1)

        x = self.conv1(x)
        x = self.relu(self.bn1(x))

        return self.dropout(x)
        
    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                
def build_wasp(bb, output_stride, BN):
    return wasp(bb, output_stride, BN)   

In [48]:
class Decoder(nn.Module):
    def __init__(self, dataset, classes, bb, BN,limbsNum):
        super(Decoder, self).__init__()
        ll_inplanes = 256
        limbsNum = limbsNum

        self.conv1 = nn.Conv2d(ll_inplanes, 48, 1, bias=False)
        self.batch1 = BN(48)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(2048, 256, 1, bias=False)
        self.last_conv = nn.Sequential(nn.Conv2d(304, 256, kernel_size=3, stride=1, padding=1, bias=False),
                                    BN(256),
                                    nn.ReLU(),
                                    nn.Dropout(0.5),
                                    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False),
                                    BN(256),
                                    nn.ReLU(),
                                    nn.Dropout(0.1),
                                    nn.Conv2d(256, classes+1, kernel_size=1, stride=1))


        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self._init_weight()


    def forward(self, x, ll_feat):

        ll_feat = self.conv1(ll_feat)
        ll_feat = self.batch1(ll_feat)
        ll_feat = self.relu(ll_feat)

        ll_feat = self.maxpool(ll_feat)

        x = F.interpolate(x, size=ll_feat.size()[2:], mode='bilinear', align_corners=True)

        x = torch.cat((x, ll_feat), dim=1)
        x = self.last_conv(x)


        return x

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

def build_decoder(dataset, num_classes, bb, BN,limbsNum):
    return Decoder(dataset ,num_classes, bb, BN,limbsNum)

In [49]:
def build_bb(bb, output_stride, BN):
    return resnet.ResNet101(output_stride, BN)

In [50]:
class unipose(nn.Module):
    def __init__(self, dataset, bb = 'resnet', output_stride = 16, num_classes = 21, sync_bn = True, freeze_bn = False, stride = 8):
        super(unipose,self).__init__()
        self.stride = stride; self.num_classes = num_classes

        BN = nn.BatchNorm2d

        self.pool_center   = nn.AvgPool2d(kernel_size=9, stride=8, padding=1)

        self.bb            =    build_bb(bb, output_stride, BN)
        self.wasp          = build_wasp(bb, output_stride, BN)
        self.decoder       = build_decoder(dataset, num_classes, bb, BN, 13)

        if freeze_bn:
            self.freeze_bn()

    def forward(self, input):
        x, ll_feat = self.bb(input)
        x = self.wasp(x)
        x = self.decoder(x, ll_feat)
        return x

    def freeze_bn(self):
        for m in self.modules():
            if isinstance(m, SynchronizedBatchNorm2d):
                m.eval()
            elif isinstance(m, nn.BatchNorm2d):
                m.eval()

    def get_1x_lr_params(self):
        modules = [self.bb]
        for i in range(len(modules)):
            for m in modules[i].named_modules():
                if isinstance(m[1], nn.Conv2d) or isinstance(m[1], SynchronizedBatchNorm2d) \
                        or isinstance(m[1], nn.BatchNorm2d):
                    for p in m[1].parameters():
                        if p.requires_grad:
                            yield p

    def get_10x_lr_params(self):
        modules = [self.aspp, self.decoder]
        for i in range(len(modules)):
            for m in modules[i].named_modules():
                if isinstance(m[1], nn.Conv2d) or isinstance(m[1], SynchronizedBatchNorm2d) \
                        or isinstance(m[1], nn.BatchNorm2d):
                    for p in m[1].parameters():
                        if p.requires_grad:
                            yield p


In [51]:
def adjust_learning_rate(optimizer, iters, base_lr, gamma, step_size, p='step', m=[1]):
    if p == 'fixed':
        lr = base_lr
    elif p == 'step':
        lr = base_lr * (gamma ** (iters // step_size))
    for i, param_group in enumerate(optimizer.param_groups):
        param_group['lr'] = lr * m[i]
    return lr

In [52]:
class Resized(object):
    
    def __init__(self, size):
        assert (isinstance(size, int) or (isinstance(size, collections.Iterable) and len(size) == 2))
        if isinstance(size, int):
            self.size = (size, size)
        else:
            self.size = size

    def get_params(img, output_size):

        height, width, _ = img.shape
        
        return (output_size[0] * 1.0 / height, output_size[1] * 1.0 / width)

    def __call__(self, img, kpt, center):

        ratio = self.get_params(img, self.size)

        return resize(img, kpt, center, ratio)
    
def normalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.sub_(m).div_(s)
    return tensor

def to_tensor(pic):
    return torch.from_numpy(pic.transpose((2, 0, 1))).float()


In [53]:
def gk(size_w, size_h, center_x, center_y, sigma):
    gridy, gridx = np.mgrid[0:size_h, 0:size_w]
    D2 = (gridx - center_x) ** 2 + (gridy - center_y) ** 2
    return np.exp(-D2 / 2.0 / sigma / sigma)


In [54]:
class coco(data.Dataset):
    def __init__(self, root_dir, sigma, is_train, transform=None):
        
        self.transformer = transform
        self.is_train    = is_train
        self.sigma       = sigma
        self.parts_num   = 17
        self.stride      = 8
        self.skeleton = [[16,14],[14,12],[17,15],[15,13],[12,13],[6,12],[7,13],[6,7],[6,8],[7,9],[8,10],[9,11],[2,3],[1,2],[1,3],[2,4],[3,5],[4,6],[5,7]]
        
        self.labels_dir  = root_dir 
        
        
        
        with open(self.labels_dir) as anno_file:
            self.anno = json.load(anno_file)
        
        if is_train == True:
            self.img_List = []
            self.images_dir  = 'train2017/'
            for idx,val in enumerate(self.anno):
                self.img_List.append(idx)
                if idx == 50000:
                    break
            print("Train images ",len(self.img_List))
                
        elif is_train == False: 
            self.img_List = []
            self.images_dir  = 'val2017/'
            for idx,val in enumerate(self.anno):
                self.img_List.append(idx)
                if idx == 4000:
                    break
            print("Val images ",len(self.img_List))
            
      
        
    def __getitem__(self, index):
        scale = 0.25
        variable = self.anno[self.img_List[index]]

        img_path  = self.images_dir + variable['file_name'] 
        keyp = [variable['keypoints'][i] for i in range(len(variable['keypoints'])) if i in (0,1,3,4,6,7,9,10,12,13,15,16,18,19,21,22,24,25,27,28,30,31,33,34,36,37,39,40,42,43,45,46,48,49)]
        keyp_ = [[keyp[i],keyp[i+1]] for i in np.arange(0,34,2)]
        points   = torch.Tensor(keyp_)
        center = {}
        img    = np.array(cv2.resize(cv2.imread(img_path),(368,368)), dtype=np.float32)
        
        center = [int(img.shape[0]/2),int(img.shape[1]/2)]

        nParts = 17
        
    
        kpt = points
            
        height, width, _ = img.shape
        heatmap = np.zeros((int(height/self.stride), int(width/self.stride), int(len(kpt)+1)), dtype=np.float32)
        for i in range(len(kpt)):
            # resize from 368 to 46
            x = int(kpt[i][0]) * 1.0 / self.stride
            y = int(kpt[i][1]) * 1.0 / self.stride
            heat_map = gk(size_h=int(height/self.stride),size_w=int(width/self.stride), center_x=x, center_y=y, sigma=self.sigma)
            heat_map[heat_map > 1] = 1
            heat_map[heat_map < 0.0099] = 0
            heatmap[:, :, i + 1] = heat_map
        
        centermap = np.zeros((int(height/self.stride), int(width/self.stride), 1), dtype=np.float32)
        center_map = gk(size_h=int(height/self.stride), size_w=int(width/self.stride), center_x=int(center[0]/self.stride), center_y=int(center[1]/self.stride), sigma=3)
        center_map[center_map > 1] = 1
        center_map[center_map < 0.0099] = 0
        centermap[:, :, 0] = center_map


        img = normalize(to_tensor(img), [128.0, 128.0, 128.0],
                                     [256.0, 256.0, 256.0])
        heatmap   = to_tensor(heatmap)
        centermap = to_tensor(centermap)

        return img, heatmap, centermap, img_path
        
    def __len__(self):
        return len(self.img_List)        
    

In [55]:
def getDataloader(dataset, train_dir, val_dir, sigma, stride, workers, batch_size,*test_dir,):
    if dataset == 'COCO':
        train_loader = torch.utils.data.DataLoader(
                                            coco(train_dir, sigma,is_train =  True,
                                            transform = Resized(368)),
                                            batch_size  = batch_size, shuffle=True,num_workers = workers, pin_memory=True)
    
        val_loader   = torch.utils.data.DataLoader(
                                            coco(val_dir, sigma, is_train = False,
                                            transform = Resized(368)),batch_size = 1, shuffle=True,num_workers = 1, pin_memory=True)
        return train_loader, val_loader 


In [56]:
def printAccuracies(mAP, AP, mPCKh, PCKh, mPCK, PCK, dataset):
    if dataset == "COCO":
        print("mPCKh: %.2f%%" % (mPCKh*100))
        print("PCKhs: %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%, %2.2f%%"\
            % (PCKh[0]*100,PCKh[1]*100,PCKh[2]*100,PCKh[3]*100,PCKh[4]*100,PCKh[5]*100,PCKh[6]*100,PCKh[7]*100,PCKh[8]*100,PCKh[9]*100,\
                PCKh[10]*100,PCKh[11]*100,PCKh[12]*100,PCKh[13]*100,PCKh[14]*100,PCKh[15]*100,PCKh[16]*100,PCKh[17]*100))

In [57]:
def get_kpts(maps, img_h = 368.0, img_w = 368.0):

    maps = maps.clone().cpu().data.numpy()
    map_6 = maps[0]

    kpts = []
    for m in map_6[1:]:
        h, w = np.unravel_index(m.argmax(), m.shape)
        x = int(w * img_w / m.shape[1])
        y = int(h * img_h / m.shape[0])
        kpts.append([x,y])
    return kpts

In [58]:
def calc_dists(preds, target, normalize):
    preds  =  preds.astype(np.float32)
    target = target.astype(np.float32)
    dists  = np.zeros((preds.shape[1], preds.shape[0]))

    for n in range(preds.shape[0]):
        for c in range(preds.shape[1]):
            if target[n, c, 0] > 1 and target[n, c, 1] > 1:
                normed_preds   =  preds[n, c, :] / normalize[n]
                normed_targets = target[n, c, :] / normalize[n]
                dists[c, n]    = np.linalg.norm(normed_preds - normed_targets)
            else:
                dists[c, n]    = -1

    return dists


def dist_acc(dists, threshold = 0.5):
    dist_cal     = np.not_equal(dists, -1)
    num_dist_cal = dist_cal.sum()

    if num_dist_cal > 0:
        return np.less(dists[dist_cal], threshold).sum() * 1.0 / num_dist_cal
    else:
        return -1


def get_max_preds(batch_heatmaps):
    batch_size = batch_heatmaps.shape[0]
    num_joints = batch_heatmaps.shape[1]
    width      = batch_heatmaps.shape[3]

    heatmaps_reshaped = batch_heatmaps.reshape((batch_size, num_joints, -1))
    idx               = np.argmax(heatmaps_reshaped, 2)
    maxvals           = np.amax(heatmaps_reshaped, 2)

    maxvals = maxvals.reshape((batch_size, num_joints, 1))
    idx     = idx.reshape((batch_size, num_joints, 1))

    preds   = np.tile(idx, (1,1,2)).astype(np.float32)

    preds[:,:,0] = (preds[:,:,0]) % width
    preds[:,:,1] = np.floor((preds[:,:,1]) / width)

    pred_mask    = np.tile(np.greater(maxvals, 0.0), (1,1,2))
    pred_mask    = pred_mask.astype(np.float32)

    preds *= pred_mask

    return preds, maxvals



def accuracy(output, target, thr_PCK, thr_PCKh, dataset, hm_type='gaussian', threshold=0.5):
    idx  = list(range(output.shape[1]))
    #print('idx=', idx)
    norm = 1.0

    if hm_type == 'gaussian':
        pred, _   = get_max_preds(output)
        target, _ = get_max_preds(target)

        h         = output.shape[2]
        w         = output.shape[3]
        norm      = np.ones((pred.shape[0], 2)) * np.array([h,w]) / 10

    dists = calc_dists(pred, target, norm)

    acc     = np.zeros((len(idx)))
    avg_acc = 0
    cnt     = 0
    visible = np.zeros((len(idx)))

    for i in range(len(idx)):
        acc[i] = dist_acc(dists[idx[i]])
        if acc[i] >= 0:
            avg_acc = avg_acc + acc[i]
            cnt    += 1
            visible[i] = 1
        else:
            acc[i] = 0

    avg_acc = avg_acc / cnt if cnt != 0 else 0

    if cnt != 0:
        acc[0] = avg_acc

    PCKh = np.zeros((len(idx)))
    avg_PCKh = 0

    if dataset == "COCO":
        headLength = np.linalg.norm(target[0,4,:] - target[0,5,:])
    elif dataset == "MPII":
        headLength = np.linalg.norm(target[0,9,:] - target[0,10,:])


    for i in range(len(idx)):
        PCKh[i] = dist_acc(dists[idx[i]], thr_PCKh*headLength)
        if PCKh[i] >= 0:
            avg_PCKh = avg_PCKh + PCKh[i]
        else:
            PCKh[i] = 0

    avg_PCKh = avg_PCKh / cnt if cnt != 0 else 0

    if cnt != 0:
        PCKh[0] = avg_PCKh


    PCK = np.zeros((len(idx)))
    avg_PCK = 0

    if dataset == "COCO":
        pelvis = [(target[0,12,0]+target[0,13,0])/2, (target[0,12,1]+target[0,13,1])/2]
        torso  = np.linalg.norm(target[0,13,:] - pelvis)
        
    elif dataset == "MPII":
        torso  = np.linalg.norm(target[0,7,0] - target[0,8,0])

    for i in range(len(idx)):
        PCK[i] = dist_acc(dists[idx[i]], thr_PCK*torso)

        if PCK[i] >= 0:
            avg_PCK = avg_PCK + PCK[i]
        else:
            PCK[i] = 0

    avg_PCK = avg_PCK / cnt if cnt != 0 else 0

    if cnt != 0:
        PCK[0] = avg_PCK


    return acc, PCK, PCKh, cnt, pred, visible


In [59]:
class Trainer(object):
    def __init__(self, pretrained,dataset,train_dir,val_dir,model_name,model_arch = 'unipose'):

        #hyperparameters
        self.train_dir    = ""
        self.val_dir      = ""
        self.dataset      = "COCO"
        self.pretrained = pretrained
        self.model_name = model_name
        self.model_arch = model_arch

        self.workers      = 1
        self.weight_decay = 0.005
        self.momentum     = 0.9
        self.batch_size   = 8
        self.lr           = 0.0005
        self.gamma        = 0.333
        self.step_size    = 13275
        self.sigma        = 3
        self.stride       = 8
        cudnn.benchmark   = True
    
        if self.dataset == "COCO":
            self.numClasses  = 17
        self.train_loader, self.val_loader = getDataloader(self.dataset, 'train_coco.json','val_coco.json', self.sigma, self.stride, self.workers, self.batch_size)
        model = unipose(self.dataset, num_classes=self.numClasses,bb='resnet',output_stride=16,sync_bn=True,freeze_bn=False, stride=self.stride)
        self.model  = model.cuda()
        self.criterion   = nn.MSELoss().cuda()
        self.optimizer   = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        self.best_model  = 12345678.9
        self.iters       = 0
        
        if self.pretrained is not None:
            checkpoint = torch.load(self.pretrained)
            p = checkpoint['state_dict']

            state_dict = self.model.state_dict()
            model_dict = {}

            for k,v in p.items():
                if k in state_dict:
                    model_dict[k] = v

            state_dict.update(model_dict)
            self.model.load_state_dict(state_dict)
            
    def training(self, epoch):
        train_loss = 0.0
        self.model.train()
        print("epoch " + str(epoch) + ':') 
        tbar = tqdm(self.train_loader)
        for i, (input, heatmap, centermap, img_path) in enumerate(tbar):
            learning_rate = adjust_learning_rate(self.optimizer, self.iters, self.lr, policy='step',
                                                 gamma=self.gamma, step_size=self.step_size)

            input_var     =     input.cuda()
            heatmap_var   =    heatmap.cuda()
            self.optimizer.zero_grad()
            heat = self.model(input_var)
            loss_heat   = self.criterion(heat,  heatmap_var)
            loss = loss_heat
            train_loss += loss_heat.item()
            loss.backward()
            self.optimizer.step()
            tbar.set_description('Train loss: %.6f' % (train_loss / ((i + 1)*self.batch_size)))
            self.iters += 1
        
           
    def validation(self, epoch):
        self.model.eval()
        tbar = tqdm(self.val_loader, desc='\r')
        val_loss = 0.0
        
        AP    = np.zeros(self.numClasses+1)
        PCK   = np.zeros(self.numClasses+1)
        PCKh  = np.zeros(self.numClasses+1)
        count = np.zeros(self.numClasses+1)

        cnt = 0
        for i, (input, heatmap, centermap, img_path) in enumerate(tbar):

            cnt += 1

            input_var     =      input.cuda()
            heatmap_var   =    heatmap.cuda()
            self.optimizer.zero_grad()

            heat = self.model(input_var)
            loss_heat   = self.criterion(heat,  heatmap_var)

            loss = loss_heat

            val_loss += loss_heat.item()

            tbar.set_description('Val loss: %.6f' % (val_loss / ((i + 1)*self.batch_size)))

            acc, acc_PCK, acc_PCKh, cnt, pred, visible = accuracy(heat.detach().cpu().numpy(), heatmap_var.detach().cpu().numpy(),0.2,0.5, self.dataset)

            AP[0]     = (AP[0]  *i + acc[0])      / (i + 1)
            PCK[0]    = (PCK[0] *i + acc_PCK[0])  / (i + 1)
            PCKh[0]   = (PCKh[0]*i + acc_PCKh[0]) / (i + 1)
            for j in range(1,self.numClasses+1):
                if visible[j] == 1:
                    AP[j]     = (AP[j]  *count[j] + acc[j])      / (count[j] + 1)
                    PCK[j]    = (PCK[j] *count[j] + acc_PCK[j])  / (count[j] + 1)
                    PCKh[j]   = (PCKh[j]*count[j] + acc_PCKh[j]) / (count[j] + 1)
                    count[j] += 1

            mAP     =   AP[1:].sum()/(self.numClasses)
            mPCK    =  PCK[1:].sum()/(self.numClasses)
            mPCKh   = PCKh[1:].sum()/(self.numClasses)

        printAccuracies(mAP, AP, mPCKh, PCKh, mPCK, PCK, self.dataset)

In [17]:
#final 10 epoch run
starter_epoch =    0
epochs        =  10
trainer = Trainer(None,'COCO', '', '', None, 'unipose' )
for epoch in range(starter_epoch, epochs):
    trainer.training(epoch)
    trainer.validation(epoch)

Train images  50001
Val images  4001


  0%|          | 0/6251 [00:00<?, ?it/s]

epoch 0:


  0%|          | 0/6251 [00:00<?, ?it/s]

mPCKh: 56.70%
PCKhs: 25.32%, 65.29%, 60.84%, 64.29%, 50.60%, 61.04%, 59.29%, 59.54%, 56.08%, 55.85%, 54.67%, 54.44%, 56.23%, 55.72%, 53.01%, 53.05%, 52.50%, 51.54%
epoch 1:


...

mPCKh: 56.95%
PCKhs: 25.40%, 65.61%, 61.22%, 64.64%, 51.08%, 61.23%, 59.50%, 59.67%, 56.52%, 56.01%, 54.86%, 54.44%, 56.39%, 55.80%, 53.34%, 53.16%, 52.64%, 52.11%
epoch 8:


  0%|          | 0/6251 [00:00<?, ?it/s]

mPCKh: 56.96%
PCKhs: 25.45%, 65.77%, 61.41%, 64.73%, 51.08%, 61.43%, 59.63%, 59.80%, 56.26%, 56.41%, 54.77%, 54.79%, 56.31%, 55.72%, 53.01%, 53.05%, 52.50%, 51.69%
epoch 9:


Val loss: 0.000511: 100%|██████████| 4001/4001 [01:28<00:00, 45.17it/s]

mPCKh: 57.06%
PCKhs: 25.47%, 65.69%, 61.31%, 64.64%, 51.20%, 61.52%, 59.56%, 59.74%, 56.52%, 56.17%, 55.24%, 54.96%, 56.55%, 55.80%, 53.23%, 53.16%, 52.78%, 51.97%


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=330e0e44-deeb-47a0-bf8a-9dcc20db6494' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>